In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from kinematics import (
    trisphere_forward_kinematics, trisphere_inverse_kinematics, rotation_x, rotation_y, rotation_z, 
    rotation_fixed_angle
)

In [ ]:
jack1_np = np.array([1150, 0, 100])
jack2_np = np.array([0, 0, 0])
jack3_np = np.array([530, 0, 970])
jack_np = (jack1_np, jack2_np, jack3_np)

jack1_np_min = jack1_np - np.array([200, 0, 0])
jack1_np_max = jack1_np + np.array([200, 0, 0])

jack2_np_min = jack2_np - np.array([0, 0, 200])
jack2_np_max = jack2_np + np.array([0, 0, 200])

jack3_np_min = jack3_np - np.array([0, 0, 200])
jack3_np_max = jack3_np + np.array([0, 0, 200])

jack_np_min = (jack1_np_min, jack2_np_min, jack3_np_min)
jack_np_max = (jack1_np_max, jack2_np_max, jack3_np_max)

c_np = np.array([300, 400, 300])

npointing = np.array([0., 0., 1.])

## Forward kinematics

In [ ]:
jack_p = (np.array([  0.        , 60.03554473, -104.02106397]), 
          np.array([ -1.89714087, 26.30424607,    0.        ]), 
          np.array([ 64.52008861, -1.49899447,    0.        ]))


c_p, c_r = trisphere_forward_kinematics(jack_p, jack_np, c_np)
print(c_p, np.rad2deg(c_r))

## Inverse kinematics

In [ ]:
def plot_arrow(ax, x0, y0, dx, dy, color):
    ax.plot(x0, y0, '.', color=color, ms=20, label='Origin of control point')
    ax.plot((x0, x0 + 100*dx), (y0, y0 + 100*dy), '-', color=color, lw=2, alpha=0.7)
    if dx or dy:
        ax.arrow(x0 + 80*dx, y0 + 80*dy, dx, dy, head_width=25, head_length=40, color=color)    

    
def visualize():
    """"""
    def visualize_plane(i, j, figsize):
        fig, ax = plt.subplots(figsize=figsize)

        # original vector
        plot_arrow(ax, c_np[i], c_np[j], npointing[i], npointing[j], 'k')
        
        # target vector
        plot_arrow(ax, c_p[i] + c_np[i], c_p[j] + c_np[j], pointing[i], pointing[j], 'r')
        
        x = [p[i] for p in jack_np]
        y = [p[j] for p in jack_np]
        ax.plot(x, y, 'x', ms=19, color='r', label='Null position')
        for p_min, p_max in zip(jack_np_min, jack_np_max):
            ax.plot((p_min[i], p_max[i]), (p_min[j], p_max[j]), '-', lw=3, color='r', alpha=0.7)
        
        x = [p[i] for p in tnp]
        y = [p[j] for p in tnp]
        ax.plot(x, y, '+', ms=19, color='g', label='Transformed null position')
        for p_min, p_max in zip(tnp_min, tnp_max):
            ax.plot((p_min[i], p_max[i]), (p_min[j], p_max[j]), '-', lw=3, color='g', alpha=0.7)

        x = [p[i] + np[i] for p, np in zip(jack_p, jack_np)]
        y = [p[j] + np[j] for p, np in zip(jack_p, jack_np)]
        ax.plot(x, y, '.', ms=19, color='b', label='New position')

        ax.set_xlabel(labels[i], fontsize=20)
        ax.set_ylabel(labels[j], fontsize=20)
        ax.legend()
        ax.grid()
        ax.set_aspect('equal')
        plt.tight_layout()
   
    labels = ('x', 'y', 'z')

    visualize_plane(0, 2, (10, 10))
    visualize_plane(0, 1, (10, 6))
    visualize_plane(2, 1, (10, 6))


def check():
    def is_colinear(p1, p2, p3):
        a1 = p1[0]*(p2[1] - p3[1]) + p2[0]*(p3[1] - p1[1]) + p3[0]*(p1[1] - p2[1])
        a2 = p1[1]*(p2[2] - p3[2]) + p2[1]*(p3[2] - p1[2]) + p3[1]*(p1[2] - p2[2])
        a3 = p1[2]*(p2[0] - p3[0]) + p2[2]*(p3[0] - p1[0]) + p3[2]*(p1[0] - p2[0])
        ret = abs(a1) < 1e-6 and abs(a2) < 1e-6 and abs(a3) < 1e-6
        if ret:
            print("passed")
        else:
            print(a1, a2, a3)
            

    for p_min, p_max, p, np in zip(tnp_min, tnp_max, jack_p, jack_np):
        is_colinear(p_min, p_max, p + np)


c_p = np.array([200., 300., 100.])
c_r = np.radians(np.array([3, 5, 2]))  # gamma, beta, alpha

rt_matrix = rotation_fixed_angle(*c_r)
pointing = (npointing * rt_matrix.T).getA()[0]

jack_p, tnp = trisphere_inverse_kinematics(c_p, c_r, c_np, jack_np)
jack_p_min, tnp_min = trisphere_inverse_kinematics(c_p, c_r, c_np, jack_np_min)
jack_p_max, tnp_max = trisphere_inverse_kinematics(c_p, c_r, c_np, jack_np_max)

for p in jack_p:
    print(p)

check() 
    
visualize()